In [2]:
import sys

# Add the weatherpy_class directory to the path
sys.path.insert(0, r'C:\Users\Administrator\Documents\weatherpy_class')


from weatherpy.data.wd_importer import BOMWeatherDataImporter, NOAAWeatherDataImporter
import logging
import traceback

# Configure logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')



In [1]:

# # def test_import_bom_data():
# """Test case for importing BOM data."""
# input_data = {
#     'stationID': '066037',
#     'dataType': 'BOM',
#     'timeZone': 'LocalTime',
#     'yearStart': 2010,
#     'yearEnd': 2020,
#     'interval': 60,
#     'saveRaw': False
# }
# try:
#     importer = BOMWeatherDataImporter(
#         station_id=input_data['stationID'],
#         time_zone=input_data['timeZone'],
#         year_start=input_data['yearStart'],
#         year_end=input_data['yearEnd'],
#         interval=input_data['interval'],
#         save_raw=input_data['saveRaw']
#     )
#     weather_data = importer.import_data()
#     logging.info("BOM data imported successfully.")
#     logging.debug(weather_data.data.head())
#     df = weather_data.data
# except Exception as e:
#     logging.error(f"Error importing BOM data: {e}")
#     logging.debug(traceback.format_exc())




# def test_import_noaa_data():
"""Test case for importing NOAA data."""
input_data = {
    'stationID': '72509014739',
    'dataType': 'NOAA',
    'timeZone': 'UTC',
    'yearStart': 2019,
    'yearEnd': 2020,
    'saveRaw': False
}
try:
    importer = NOAAWeatherDataImporter(
        station_id=input_data['stationID'],
        time_zone=input_data['timeZone'],
        year_start=input_data['yearStart'],
        year_end=input_data['yearEnd'],
        save_raw=input_data['saveRaw']
    )
    weather_data = importer.import_data()
    df = weather_data.data
    logging.info("NOAA data imported successfully.")
    logging.debug(weather_data.data.head())
except Exception as e:
    logging.error(f"Error importing NOAA data: {e}")
    logging.debug(traceback.format_exc())


# Run test cases
# logging.info('\n=== Running Test Cases ===')
# test_import_bom_data()
# test_import_noaa_data()
# logging.info('=== Test Cases Complete ===') 

2025-03-15 06:24:14,194 - DEBUG - Starting new HTTPS connection (1): www.ncei.noaa.gov:443


Importing NOAA data for station 72509014739 from 2019 to 2020
API date range (5 extra days either end): 2018-12-25T00:00:00 to 2021-01-05T23:59:00
API attempt 1/3


2025-03-15 06:24:15,680 - DEBUG - https://www.ncei.noaa.gov:443 "GET /access/services/data/v1?dataset=global-hourly&stations=72509014739&dataTypes=WND,CIG,VIS,TMP,DEW,SLP,LONGITUDE,LATITUDE,ELEVATION,GA1,AA2&startDate=2018-12-25T00:00:00&endDate=2021-01-05T23:59:00&format=json HTTP/1.1" 200 None
2025-03-15 06:24:22,342 - INFO - NOAA data imported successfully.
2025-03-15 06:24:22,383 - DEBUG -                                           LocalTime REPORT_TYPE  \
UTC                                                               
2019-01-01 00:00:00+00:00 2018-12-31 19:00:00-05:00       FM-12   
2019-01-01 00:54:00+00:00 2018-12-31 19:54:00-05:00       FM-15   
2019-01-01 01:54:00+00:00 2018-12-31 20:54:00-05:00       FM-15   
2019-01-01 02:38:00+00:00 2018-12-31 21:38:00-05:00       FM-16   
2019-01-01 02:54:00+00:00 2018-12-31 21:54:00-05:00       FM-15   

                          QUALITY_CONTROL  LONGITUDE  ELEVATION  SOURCE  \
UTC                                                       

In [2]:
import logging
import sys
import pandas as pd
import numpy as np
import os
import traceback
from datetime import datetime
import pytz

# Add the weatherpy_class directory to the path
sys.path.insert(0, r'C:\Users\Administrator\Documents\weatherpy_class')

import weatherpy as wp
from weatherpy.data.wd_base import WeatherData
from weatherpy.data.wd_importer import BOMWeatherDataImporter, NOAAWeatherDataImporter
from weatherpy.data.wd_stations import WeatherStationDatabase

# Import the legacy weatherpy
import weatherpy_legacy as wpl

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def compare_dataframes(df1, df2):
    """Compare two DataFrames and print differences in a concise way."""
    if df1 is None or df2 is None:
        print("Cannot compare DataFrames: one or both are None")
        return
        
    if df1.equals(df2):
        print("✓ Data outputs are identical.")
        return
    
    print("✗ Data outputs differ.")
    
    # Compare column differences
    cols1 = set(df1.columns)
    cols2 = set(df2.columns)
    col_diff1 = cols1 - cols2
    col_diff2 = cols2 - cols1
    
    if col_diff1 or col_diff2:
        print("  Column differences:")
        if col_diff1:
            print(f"  - Only in legacy DataFrame: {', '.join(sorted(col_diff1))}")
        if col_diff2:
            print(f"  - Only in OOP DataFrame: {', '.join(sorted(col_diff2))}")
    
    # Compare shapes
    if df1.shape != df2.shape:
        print(f"  Shape difference: {df1.shape} vs {df2.shape}")
    
    # Check for NaN values
    if not df1.isna().equals(df2.isna()):
        print("  NaN value differences detected.")
    
    # Check for data type consistency
    if not df1.dtypes.equals(df2.dtypes):
        print("  Data type differences detected:")
        # Instead of using compare() which requires identical labels,
        # manually compare the dtypes for common columns
        common_cols = set(df1.columns) & set(df2.columns)
        dtype_diff = {}
        
        for col in common_cols:
            if df1[col].dtype != df2[col].dtype:
                dtype_diff[col] = {'legacy': df1[col].dtype, 'oop': df2[col].dtype}
        
        if dtype_diff:
            print("  Common columns with different dtypes:")
            for col, types in dtype_diff.items():
                print(f"    {col}: {types['legacy']} vs {types['oop']}")
        
        # Show dtypes for columns that exist only in one DataFrame
        only_df1_cols = set(df1.columns) - set(df2.columns)
        if only_df1_cols:
            print("  Columns only in legacy DataFrame:")
            for col in sorted(only_df1_cols):
                print(f"    {col}: {df1[col].dtype}")
        
        only_df2_cols = set(df2.columns) - set(df1.columns)
        if only_df2_cols:
            print("  Columns only in OOP DataFrame:")
            for col in sorted(only_df2_cols):
                print(f"    {col}: {df2[col].dtype}")
    
    # Check for index consistency
    if not df1.index.equals(df2.index):
        print("  Index differences detected.")
    
    # For common columns, compare values
    common_cols = cols1 & cols2
    if common_cols:
        diff_cols = []
        for col in common_cols:
            if not df1[col].equals(df2[col]):
                diff_cols.append(col)
        
        if diff_cols:
            print(f"  Value differences in {len(diff_cols)} columns: {', '.join(sorted(diff_cols))}")

def compare_data_import(stationID, dataType, timeZone, yearStart, yearEnd, interval, verbose=False):
    """
    Compare data import between legacy and OOP versions.
    
    Parameters
    ----------
    stationID : str
        Station ID.
    dataType : str
        Data type (BOM or NOAA).
    timeZone : str
        Time zone (UTC or LocalTime).
    yearStart : int
        Start year.
    yearEnd : int
        End year.
    interval : str
        Data interval (60minute or daily).
    verbose : bool, optional
        Whether to print verbose output. Default is False.
    
    Returns
    -------
    tuple
        (legacy_data, oop_data)
    """
    print(f"Comparing {dataType} import for station {stationID} ({timeZone})...")
    
    # Import legacy data
    print("Importing legacy data...")
    try:
        # Get station info
        if dataType == 'BOM':
            import weatherpy_legacy.data.stations as stations
            station_info = stations.station_info(stationID, printed=True)
            print('legacy timezone: ', timeZone)
            # Import data
            print("\n-----------------------------------------------------\n")
            # Use the main import_data function from weatherpy_legacy
            data_old, _, _ = wpl.import_data(
                stationID,
                dataType,
                timeZone,
                yearStart,
                yearEnd,
                interval,
                save_raw=False
            )
        elif dataType == 'NOAA':
            import weatherpy_legacy.data.stations as stations
            station_info = stations.station_info(stationID, printed=True)
            
            # Import data
            print("\n-----------------------------------------------------\n")
            # Use the main import_data function from weatherpy_legacy
            
            data_old, _, _ = wpl.import_data(
                stationID,
                dataType,
                timeZone,
                yearStart,
                yearEnd,
                interval,
                save_raw=False
            )
        else:
            print(f"Error: Unsupported data type: {dataType}")
            return None, None
    except Exception as e:
        print(f"Error importing legacy data: {e}")
        import traceback
        traceback.print_exc()
        data_old = None
    
    # Import OOP data
    print("\nImporting OOP data...")
    try:
        # Get station info
        if dataType == 'BOM':
            from weatherpy.data.wd_stations import WeatherStationDatabase
            try:
                station_db = WeatherStationDatabase('BOM')
                station_info = station_db.get_station_info(stationID)
            except Exception as e:
                print(f"Warning: Could not get station information: {e}")
            
            importer = wp.BOMWeatherDataImporter(
                station_id=stationID,
                data_type=dataType,
                time_zone=timeZone,
                year_start=yearStart,
                year_end=yearEnd,
                interval=interval
            )

            weather_data = importer.import_data()
            data_class = weather_data.data
            
        elif dataType == 'NOAA':
            from weatherpy.data.wd_stations import WeatherStationDatabase
            try:
                station_db = WeatherStationDatabase('NOAA')
                station_info = station_db.get_station_info(stationID)
            except Exception as e:
                print(f"Warning: Could not get station information: {e}")
            
            importer = wp.NOAAWeatherDataImporter(
                    station_id=stationID,
                    data_type=dataType,
                    time_zone=timeZone,
                    year_start=yearStart,
                    year_end=yearEnd,
                    interval=interval
                )
                
            weather_data = importer.import_data()
            data_class = weather_data.data
        else:
            print(f"Error: Unsupported data type: {dataType}")
            return None, None
    except Exception as e:
        print(f"Error importing OOP data: {e}")
        import traceback
        traceback.print_exc()
        data_class = None
    
    # Compare results
    if data_old is not None and data_class is not None:
        print("\nComparing import results:")
        compare_dataframes(data_old, data_class)
    
    return data_old, data_class

# Example usage
input_data_1 = {
    'stationID': '066037',  # Using a valid station ID from the database (WYNDHAM AERO)
    'dataType': 'BOM',
    'timeZone': 'LocalTime',
    'yearStart': 2018,
    'yearEnd': 2020,
    'interval': 60,
    'verbose': False  # Set to True for more detailed output
}

input_data_2 = {
    'stationID': '72509014739',
    'dataType': 'NOAA',
    'timeZone': 'LocalTime',
    'yearStart': 2018,
    'yearEnd': 2020,
    'interval': 30,
    'verbose': False  # Set to True for more detailed output
}

# Run the comparison
print('\n=== WeatherPy Import Comparison ===')
print('=' * 30)


# Run BOM import comparison
print('\nRunning BOM import comparison...')
# data_old_bom, data_class_bom = compare_data_import(**input_data_1)

# Uncomment to run NOAA import comparison
# print('\nRunning NOAA import comparison...')
data_old_noaa, data_class_noaa = compare_data_import(**input_data_2)

# print('\n=== Comparison Complete ===')

# def test_import_bom_data():
#     """Test case for importing BOM data."""
#     input_data = {
#         'stationID': '066037',
#         'dataType': 'BOM',
#         'timeZone': 'LocalTime',
#         'yearStart': 2010,
#         'yearEnd': 2020,
#         'interval': 60
#     }
#     try:
#         importer = BOMWeatherDataImporter(
#             station_id=input_data['stationID'],
#             time_zone=input_data['timeZone'],
#             year_start=input_data['yearStart'],
#             year_end=input_data['yearEnd'],
#             interval=input_data['interval']
#         )
#         weather_data = importer.import_data()
#         print("BOM data imported successfully.")
#         # print(weather_data.data.head())
#     except Exception as e:
#         print(f"Error importing BOM data: {e}")


# def test_import_noaa_data():
#     """Test case for importing NOAA data."""
#     input_data = {
#         'stationID': '72509014739',
#         'dataType': 'NOAA',
#         'timeZone': 'LocalTime',
#         'yearStart': 2010,
#         'yearEnd': 2020,
#         'interval': 60
#     }
#     try:
#         importer = NOAAWeatherDataImporter(
#             station_id=input_data['stationID'],
#             time_zone=input_data['timeZone'],
#             year_start=input_data['yearStart'],
#             year_end=input_data['yearEnd'],
#             interval=input_data['interval']
#         )
#         weather_data = importer.import_data()
#         print("NOAA data imported successfully.")
#         # print(weather_data.data.head())
#     except Exception as e:
#         print(f"Error importing NOAA data: {e}")


# # Run test cases
# print('\n=== Running Test Cases ===')
# test_import_bom_data()
# # test_import_noaa_data()
# print('=== Test Cases Complete ===') 

2025-03-15 06:24:22,649 - DEBUG - matplotlib data path: c:\Users\Administrator\Documents\weatherpy_class\.venv\Lib\site-packages\matplotlib\mpl-data
2025-03-15 06:24:22,658 - DEBUG - CONFIGDIR=C:\Users\Administrator\.matplotlib
2025-03-15 06:24:22,894 - DEBUG - interactive is False
2025-03-15 06:24:22,895 - DEBUG - platform is win32
2025-03-15 06:24:23,011 - DEBUG - CACHEDIR=C:\Users\Administrator\.matplotlib
2025-03-15 06:24:23,017 - DEBUG - Using fontManager instance from C:\Users\Administrator\.matplotlib\fontlist-v390.json



=== WeatherPy Import Comparison ===

Running BOM import comparison...
Comparing NOAA import for station 72509014739 (LocalTime)...
Importing legacy data...

Station ID: 72509014739

Station Name:     GEN E L LOGAN INTERNATIONAL A
Country:          United States
State:            MA
Timezone:         America/New_York
Timezone Ofset:   UTC -05:00
Coordinate:       (42.361, -71.01)
Altitude:         3.3 m
Years Active:     21/11/1943 - 01/02/2023
Source:           NOAA
M Correction Factors: N/A

-----------------------------------------------------


-----------------------------------------------------


Station ID: 72509014739

Station Name:     GEN E L LOGAN INTERNATIONAL A
Country:          United States
State:            MA
Timezone:         America/New_York
Timezone Ofset:   UTC -05:00
Coordinate:       (42.361, -71.01)
Altitude:         3.3 m
Years Active:     21/11/1943 - 01/02/2023
Source:           NOAA
M Correction Factors: N/A

------------------------------------------------

2025-03-15 06:24:26,182 - DEBUG - Starting new HTTPS connection (1): www.ncei.noaa.gov:443


NOAA Server
	API attempt 1/10 : 

2025-03-15 06:24:27,060 - DEBUG - https://www.ncei.noaa.gov:443 "GET /access/services/data/v1?dataset=global-hourly&stations=72509014739&dataTypes=WND,CIG,VIS,TMP,DEW,SLP,LONGITUDE,LATITUDE,ELEVATION,GA1,AA2&startDate=2017-12-25T00:00:00&endDate=2021-01-05T23:59:00&format=json HTTP/1.1" 200 None


...

	--Data is imported--


Importing OOP data...


2025-03-15 06:24:39,195 - DEBUG - Starting new HTTPS connection (1): www.ncei.noaa.gov:443


Importing NOAA data for station 72509014739 from 2018 to 2020
API date range (5 extra days either end): 2017-12-25T00:00:00 to 2021-01-05T23:59:00
API attempt 1/3


2025-03-15 06:24:39,997 - DEBUG - https://www.ncei.noaa.gov:443 "GET /access/services/data/v1?dataset=global-hourly&stations=72509014739&dataTypes=WND,CIG,VIS,TMP,DEW,SLP,LONGITUDE,LATITUDE,ELEVATION,GA1,AA2&startDate=2017-12-25T00:00:00&endDate=2021-01-05T23:59:00&format=json HTTP/1.1" 200 None



Comparing import results:
✗ Data outputs differ.
  Column differences:
  - Only in legacy DataFrame: DewPointTemp, QCName, QCWindDir, ReportType, Temperature, WindDir
  - Only in OOP DataFrame: AA1, AA3, AA4, CALL_SIGN, CavokCode, CeilingDetCode, DewPointTemperature, DryBulbTemperature, ELEVATION, GA2, GA3, GA4, GA5, GA6, LATITUDE, LONGITUDE, QCCloudHgt, QCDewPoint, QCSeaLevelPressure, QCTemperature, QCVisVar, QCVisibility, QCWindDirection, QUALITY_CONTROL, REPORT_TYPE, SOURCE, VisibilityVarCode, WindDirection
  Shape difference: (42127, 21) vs (42127, 43)
  NaN value differences detected.
  Data type differences detected:
  Columns only in legacy DataFrame:
    DewPointTemp: float64
    QCName: object
    QCWindDir: object
    ReportType: object
    Temperature: float64
    WindDir: int64
  Columns only in OOP DataFrame:
    AA1: float64
    AA3: float64
    AA4: float64
    CALL_SIGN: object
    CavokCode: object
    CeilingDetCode: object
    DewPointTemperature: float64
    DryBul

In [3]:
# print(data_class_noaa.columns)
# print(data_old_noaa.columns)

print(data_class_noaa['WindSpeed'].equals(data_old_noaa['WindSpeed']))
print(data_class_noaa['WindSpeed'].dtype)
print(data_old_noaa['WindSpeed'].dtype)
print(data_class_noaa['DryBulbTemperature'].tail())
print(data_old_noaa['Temperature'].tail())

True
float64
float64
LocalTime
2020-12-31 22:00:00-05:00      2.2
2020-12-31 22:54:00-05:00      1.7
2020-12-31 23:54:00-05:00      0.6
2020-12-31 23:59:00-05:00    999.9
2020-12-31 23:59:00-05:00    999.9
Name: DryBulbTemperature, dtype: float64
LocalTime
2020-12-31 22:00:00-05:00      2.2
2020-12-31 22:54:00-05:00      1.7
2020-12-31 23:54:00-05:00      0.6
2020-12-31 23:59:00-05:00    999.9
2020-12-31 23:59:00-05:00    999.9
Name: Temperature, dtype: float64


In [4]:
# 1. Check if there are any differences at all
print("Are all values equal?", data_class_noaa['WindSpeed'].equals(data_old_noaa['WindSpeed']))

# 2. Calculate the absolute difference between the two columns
diff = (data_class_noaa['WindSpeed'] - data_old_noaa['WindSpeed']).abs()

# 3. Check if there are any non-zero differences
print("Number of different values:", (diff > 0).sum())

# 4. Find the maximum difference
print("Maximum difference:", diff.max())

# 5. Find the indices where differences occur
diff_indices = diff[diff > 0].index

# 6. Show the first few differences
if len(diff_indices) > 0:
    print("\nFirst 5 differences:")
    for idx in diff_indices[:5]:
        print(f"Index: {idx}")
        print(f"  Class value: {data_class_noaa.loc[idx, 'WindSpeed']}")
        print(f"  Legacy value: {data_old_noaa.loc[idx, 'WindSpeed']}")
        print(f"  Difference: {data_class_noaa.loc[idx, 'WindSpeed'] - data_old_noaa.loc[idx, 'WindSpeed']}")
        print()

# 7. Check for very small floating-point differences
print("Are values approximately equal?", np.allclose(
    data_class_noaa['WindSpeed'].fillna(0), 
    data_old_noaa['WindSpeed'].fillna(0),
    rtol=1e-5, atol=1e-8
))

# 8. Check if the difference is due to NaN values in different places
print("NaN count in class data:", data_class_noaa['WindSpeed'].isna().sum())
print("NaN count in legacy data:", data_old_noaa['WindSpeed'].isna().sum())

# 9. Check if NaNs are in the same positions
print("NaNs in same positions?", 
      data_class_noaa['WindSpeed'].isna().equals(data_old_noaa['WindSpeed'].isna()))

# 10. If there are many differences, create a histogram of differences
if (diff > 0).sum() > 10:
    import matplotlib.pyplot as plt
    plt.figure(figsize=(10, 6))
    plt.hist(diff[diff > 0], bins=50)
    plt.title('Histogram of WindSpeed Differences')
    plt.xlabel('Absolute Difference')
    plt.ylabel('Frequency')
    plt.show()

Are all values equal? True
Number of different values: 0
Maximum difference: 0.0
Are values approximately equal? True
NaN count in class data: 0
NaN count in legacy data: 0
NaNs in same positions? True


In [22]:
data_class_noaa.columns


Index(['UTC', 'REPORT_TYPE', 'QUALITY_CONTROL', 'LONGITUDE', 'ELEVATION',
       'SOURCE', 'LATITUDE', 'CALL_SIGN', 'WindDirection', 'QCWindDirection',
       'WindType', 'WindSpeed', 'QCWindSpeed', 'CloudHgt', 'QCCloudHgt',
       'CeilingDetCode', 'CavokCode', 'Visibility', 'QCVisibility',
       'VisibilityVarCode', 'QCVisVar', 'DryBulbTemperature', 'QCTemperature',
       'DewPointTemperature', 'QCDewPoint', 'SeaLevelPressure',
       'QCSeaLevelPressure', 'CloudOktas', 'GA2', 'GA3', 'GA4', 'GA5', 'GA6',
       'AA1', 'RainCumulative', 'AA3', 'AA4', 'OC1_0', 'MW1_0', 'MW1_1',
       'AJ1_0', 'RH1_2', 'GA1_0'],
      dtype='object')

In [6]:
input_data = {
    'stationID': '72509014739',
    'dataType': 'NOAA',
    'timeZone': 'LocalTime',
    'yearStart': 2018,
    'yearEnd': 2020,
    'interval': 30,
    'verbose': False  # Set to True for more detailed output
}


importer = NOAAWeatherDataImporter(
        station_id=input_data['stationID'],
        time_zone=input_data['timeZone'],
        year_start=input_data['yearStart'],
        year_end=input_data['yearEnd'],
    )

weather_data = importer.import_data()

2025-03-15 12:05:18,368 - DEBUG - Starting new HTTPS connection (1): www.ncei.noaa.gov:443


Importing NOAA data for station 72509014739 from 2018 to 2020
API date range (5 extra days either end): 2017-12-25T00:00:00 to 2021-01-05T23:59:00
API attempt 1/3


2025-03-15 12:05:19,169 - DEBUG - https://www.ncei.noaa.gov:443 "GET /access/services/data/v1?dataset=global-hourly&stations=72509014739&dataTypes=WND,CIG,VIS,TMP,DEW,SLP,LONGITUDE,LATITUDE,ELEVATION,GA1,AA2&startDate=2017-12-25T00:00:00&endDate=2021-01-05T23:59:00&format=json HTTP/1.1" 200 None


In [9]:
print(weather_data.columns)


['UTC', 'WindDirection', 'WindSpeed', 'SeaLevelPressure', 'DryBulbTemperature', 'DewPointTemperature', 'RainCumulative', 'Visibility', 'LONGITUDE', 'LATITUDE', 'ELEVATION']


In [8]:
weather_data.unify(additional_columns=['LONGITUDE','LATITUDE','ELEVATION'])

In [11]:
weather_data.operations_log


[{'timestamp': '2025-03-15T12:05:28.278936',
  'operation': 'Initialize',
  'parameters': {'data_type': None, 'interval': None}},
 {'timestamp': '2025-03-15T12:06:08.946425',
  'operation': 'UpdateData',
  'parameters': {'shape': (42127, 11)}},
 {'timestamp': '2025-03-15T12:06:08.946425',
  'operation': 'Unify',
  'parameters': {'columns_before': ['UTC',
    'REPORT_TYPE',
    'QUALITY_CONTROL',
    'LONGITUDE',
    'ELEVATION',
    'SOURCE',
    'LATITUDE',
    'CALL_SIGN',
    'WindDirection',
    'QCWindDirection',
    'WindType',
    'WindSpeed',
    'QCWindSpeed',
    'CloudHgt',
    'QCCloudHgt',
    'CeilingDetCode',
    'CavokCode',
    'Visibility',
    'QCVisibility',
    'VisibilityVarCode',
    'QCVisVar',
    'DryBulbTemperature',
    'QCTemperature',
    'DewPointTemperature',
    'QCDewPoint',
    'SeaLevelPressure',
    'QCSeaLevelPressure',
    'CloudOktas',
    'GA2',
    'GA3',
    'GA4',
    'GA5',
    'GA6',
    'AA1',
    'RainCumulative',
    'AA3',
    'AA4',
  